In [2]:
import numpy as np
import pandas as pd

import os

from matplotlib import pyplot as plt
from matplotlib.pyplot import specgram

import librosa
import librosa.display
import IPython.display as ipd
from IPython.display import Audio

from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.model_selection import GridSearchCV

from tensorflow import keras
from keras import layers
from keras.layers import LSTM, Dense, Dropout

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Aprendizaje de maquinas/Proyecto

In [ ]:
paths=[]
labels=[]

for dirname, _, filenames in os.walk('DataSets/Emotions/'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))
        label = filename[::-1].split('.')[0][::-1]
        name = dirname[::].split('/')[2]
        name_complete = name+'.'+label
        labels.append(name_complete.lower())

In [ ]:
data = pd.DataFrame({'speech':paths,'label':labels})
data.sample(10)

In [ ]:
label_encoder = preprocessing.LabelEncoder()
data['label'] = data['label'].apply(lambda x: str(x).split('.wav')[0])
data['label']= label_encoder.fit_transform(data['label'])

In [ ]:
data.head()

In [ ]:
data['label'].unique().tolist()

In [ ]:
def padding(array, xx, yy):
    """
    :param array: numpy array
    :param xx: desired height
    :param yy: desirex width
    :return: padded array
    """

    h = array.shape[0]
    w = array.shape[1]

    a = (xx - h) // 2
    aa = xx - a - h

    b = (yy - w) // 2
    bb = yy - b - w

    return np.pad(array, pad_width=((a, aa), (b, bb)), mode='constant')

In [ ]:
df_in=data['speech']

In [ ]:
def get_features(df_in):
  features=[]
  for index in range(0,len(df_in)):
      filename = df_in[index]
      y, sr = librosa.load(filename,duration=3,offset=0.5)
      #datal = np.array([padding(librosa.feature.mfcc(y=y,sr=sr,n_mfcc=40),1,100)])
      hop_length = 512 #the default spacing between frames
      n_fft = 12816 #number of samples
      datal = np.array([padding(librosa.feature.mfcc(y,n_fft=n_fft,hop_length=hop_length,n_mfcc=128),128,400)])
      features.append(datal)
      output=np.concatenate(features,axis=0)
  return(np.array(output))


In [ ]:
X=get_features(df_in)

In [ ]:
X.shape

In [ ]:
X = np.array((X-np.min(X))/(np.max(X)-np.min(X)))
X = X/np.std(X)

In [ ]:
y = np.asarray(data['label'])

In [ ]:
X

In [ ]:
print(f'Dimensiones mfcc:{X.shape}')
print(f'Dimensiones label:{y.shape}')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=123)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=123)

In [ ]:
input_shape=(128,400)
model = keras.Sequential()
model.add(LSTM(128,input_shape=input_shape))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(7, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='SparseCategoricalCrossentropy',metrics=['acc'])

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=72,validation_data=(X_val, y_val))

In [ ]:
y_pred_test=model.predict(X_test)

In [ ]:
history_dict=history.history
loss_values=history_dict['loss']
acc_values=history_dict['acc']
val_loss_values = history_dict['val_loss']
val_acc_values=history_dict['val_acc']
epochs=range(1,51)
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(15,5))
ax1.plot(epochs,loss_values,'co',label='Training Loss')
ax1.plot(epochs,val_loss_values,'m', label='Validation Loss')
ax1.set_title('Training and validation loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.legend()
ax2.plot(epochs,acc_values,'co', label='Training accuracy')
ax2.plot(epochs,val_acc_values,'m',label='Validation accuracy')
ax2.set_title('Training and validation accuracy')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax2.legend()

In [ ]:
print(accuracy_score(y_true=y_test,y_pred=np.argmax(y_pred_test),axis=1))
print(classification_report(y_test,y_pred_test)) 
matrizr = confusion_matrix(y_test,y_pred_test)
plt.figure(figsize=(10,8))
sns.heatmap(matrizr, annot=True, fmt="d");